In [1]:
import os
from os import chdir
print(os.getcwd())
chdir('C:\\Users\\eric\\Desktop')

C:\Users\eric


In [2]:
import pandas as pd
import csv
import time
import itertools
import numpy as np


In [81]:
patent_df=pd.read_csv("selfdriving_patent_0329.csv")

In [82]:
patent_df.head()


,ipc_list,patent_abstract,patent_date,patent_nmber,patent_title
0,"G01C, G05D, G06Q, G07B",A delivery server may generate two separate se...,2018,S10002466,Method and system for providing atonomos car e...
1,"G01C, G05D, G08G, H04B, H04W","Methods, apparatses, and non-transitory compte...",2018,S10115305,Optimizing atonomos cars driving time and ser ...
2,"B61D, B61L, E01B",An atonomos ballast consist for nloading balla...,2018,S10138603,Atonomos ballast nloading consist
3,B61J,An active rail transport system for the atomat...,2012,S8146506,Active rail transport system
4,"A61M, G05D",An atonomos vehicle is provided that incldes a...,2015,S9081382,Atonomos vehicle comprising extracorporeal blo...


In [22]:
patent_df = patent_df.drop('Unnamed: 11',1)

In [83]:
patent_df=patent_df.rename(columns={"ipc_list":"ipc","patent_date":"filled_date"})
print(patent_df.head())

                            ipc  \
0        G01C, G05D, G06Q, G07B   
1  G01C, G05D, G08G, H04B, H04W   
2              B61D, B61L, E01B   
3                          B61J   
4                    A61M, G05D   

                                     patent_abstract  filled_date  \
0  A delivery server may generate two separate se...         2018   
1  Methods, apparatses, and non-transitory compte...         2018   
2  An atonomos ballast consist for nloading balla...         2018   
3  An active rail transport system for the atomat...         2012   
4  An atonomos vehicle is provided that incldes a...         2015   

  patent_nmber                                       patent_title  
0    S10002466  Method and system for providing atonomos car e...  
1    S10115305  Optimizing atonomos cars driving time and ser ...  
2    S10138603                  Atonomos ballast nloading consist  
3     S8146506                       Active rail transport system  
4     S9081382  Atonomos vehicle c

In [84]:
data=patent_df


In [85]:
import csv
import time
import itertools
import numpy as np
# node edge
def MakingEdgeNodelist(keyword_vec,sep_sign,year):
    # (1) Keyword list (data set for combination)
    keywords_list =[]
    for words in keyword_vec:
        #print("words :", words)
        split_words = words.split(sep_sign) # sep_sign으로 구분된 데이터를 분리하여
        split_words = [ x.replace(" ","") for x in split_words]                                                # 리스트로 저장
        soted_words = sorted(split_words) # ordering
        keywords_list.append(soted_words)

    # (2) Combination
    comb_list = []
    for i in keywords_list:
        for subset in itertools.combinations(i, 2):
            if subset[0] != subset[1]: # 같은 단어의 컴비네이션 제외
                comb_list.append(subset)

    # (3) Make dict & count
    comb_count = {}
    for combi in comb_list:
        comb_count[combi] = comb_count.get(combi,0)+1

    # (4) dic to df
    comb_df = pd.DataFrame()

    ## (4-1) dic_key to list
    comb_keylist = []
    for key in comb_count.keys():
        combkey_tolist = list(key)
        comb_keylist.append(combkey_tolist)

    ## (4-2) dic_key split
    source_list = []
    target_list = []
    for i in range(len(comb_keylist)):
        source = comb_keylist[i][0]
        target = comb_keylist[i][1]

        source_list.append(source)
        target_list.append(target)

    comb_df['Source_Label'] = source_list
    comb_df['Target_Label'] = target_list
    comb_df['weight'] = comb_count.values()
    comb_df['year'] = year

    comb_df = comb_df.sort_values('Source_Label',axis=0).reset_index(drop=True)

    # (5) node data
    total_tech_list = []
    for tech_li in keywords_list:
        tech = list(set(tech_li))
        total_tech_list.extend(tech)
    # node_data = pd.concat([comb_df['Source_Label'],comb_df['Target_Label']],axis=0) # 총 출현 기술 ; 잘못됨을 확인함

    node_df = pd.DataFrame()

    node_count = {}
    for node in total_tech_list:
        node_count[node] = node_count.get(node,0)+1

    node_df['label'] = node_count.keys()
    node_df['count'] = node_count.values()
    node_df['id'] = node_df.index
    node_df['year'] = year

    # (6) make id
    node_id = node_df[['label','id']]
    node_id.columns = ['Source_Label','Source']

    merge_sid = pd.merge(comb_df,node_id,how = 'inner')

    node_id.columns = ['Target_Label','Target']

    merge_tid = pd.merge(merge_sid,node_id,how = 'inner') # merge 과정에서 순서가 바뀜

    edge_df = merge_tid.sort_values(['Source_Label','Target_Label'])

    edge_df['Type'] = ['Undirected']*len(edge_df)

    #edge_cols = ['Source','Target','weight','Type','Source_Label','Target_Label']
    #edgelist = edgelist[edge_cols]

    return(edge_df,node_df)


# ipc 개수 2 이상인 데이터 필터링
ipc_len = [ len(x.split(",")) for x in data.ipc]
data['ipc_len'] = ipc_len

dataset = data.loc[data.ipc_len>1]

# edge / node list
edge_list = pd.DataFrame()
node_list = pd.DataFrame()

s_time = time.time()
for year in range(2007,2019):
    doc = dataset.loc[dataset['filled_date']== year,'ipc']
    
    edge_part,node_part = MakingEdgeNodelist(doc,",",year)
    
    edge_list = pd.concat([edge_list,edge_part],axis=0)
    
    node_list = pd.concat([node_list,node_part],axis=0)
    
e_time = time.time()
print("Execution_time :",e_time - s_time)

#

print(node_list.head())
print(edge_list.head())

('Execution_time :', 0.17799997329711914)
  label  count  id  year
0  G09D      1   0  2007
1  G09B      1   1  2007
2  G05B      2   2  2007
3  G01R      1   3  2007
4  G05D      1   4  2007
  Source_Label Target_Label  weight  year  Source  Target        Type
0         A21B         A47J       1  2007      45      26  Undirected
1         A47B         E05D       1  2007      22      12  Undirected
2         B01F         G09B       1  2007      56       1  Undirected
3         B01F         G09D       1  2007      56       0  Undirected
5         B02B         B02C       3  2007      24      25  Undirected


In [86]:
node_list.to_csv("node_list2018_eric.csv",index=False)
edge_list.to_csv("edge_list2018_eric.csv",index=False)

In [87]:
node_data = pd.read_csv("node_list2018_eric.csv")
edge_data = pd.read_csv("edge_list2018_eric.csv")

# year form trans
node_data['time'] = [ str(x)  for x in node_data.year]
edge_data['time'] = [ str(x)  for x in edge_data.year]

# =========== tiemstamp form ================= #

# [1] node

# ===== 1. time stamp format 변환 ===== #
node_dict = {} 
node_dict2 = {}
n_year_list = []
n_label_list = list(set(node_data['label']))

# time_stamp
for i in n_label_list:
    ny = node_data[node_data['label']==i]['time']
    node_dict[i] = list(ny)
# time_score_stamp
n_time_score = []
for i in range(len(node_data)):
    ny = node_data['time'].iloc[i]
    nc = node_data['count'].iloc[i]
    nt = str(ny) + "&" + str(nc)
    n_time_score.append(nt)
node_data['time_score'] = n_time_score

for i in n_label_list:
    nt = node_data[node_data['label']==i]['time_score']
    re_nt = [ "[" + x + "]" for x in nt]
    node_dict2[i] = list(re_nt)

# ===== 2. 변환한 속성 merge ==== #
n_df = pd.DataFrame()
n_df['label'] = node_dict.keys()
n_df['time_stamp'] = node_dict.values()
n_df['time_score_stamp'] = node_dict2.values()

# ===== 3. 타입변환 및 replace ==== #
# 1) time stamp
n_df['time_stamp'] = [ "<" + str(x) + ">" for x in n_df['time_stamp']]
n_df['time_stamp'] = [ x.replace("'","") for x in n_df['time_stamp']]
print(n_df['time_stamp'][0])

# 1) time socre stamp
n_df['time_score_stamp'] = [ str(x)[1:-2] for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ "<" + x + ">" for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ x.replace("'","") for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ x.replace(",",";") for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ x.replace("&",",") for x in n_df['time_score_stamp']]

print(n_df['time_score_stamp'][0])

# 2) string replace



<[2016]>
<[2016,1]>


In [89]:
    
# [2] edge
# ===== 0. name column 생성 ===== #
edge_data['t_label'] = edge_data['Source_Label'] + "&" + edge_data['Target_Label']

# ===== 1. time stamp format 변환 ===== #
edge_dict = {}
edge_dict2 = {}
e_year_list = []
e_label_list = list(set(edge_data['t_label']))

# time_stamp
for i in e_label_list:
        ey = edge_data[edge_data['t_label']==i]['time']
        edge_dict[i] = list(ey)


In [90]:
    
# time_score_stamp
e_time_score = []
for i in range(len(edge_data)):
        ey = edge_data['time'].iloc[i]
        ec = edge_data['weight'].iloc[i]
        et = str(ey) + "&" + str(ec)
        e_time_score.append(et)
edge_data['time_score'] = e_time_score

for i in e_label_list:
        et = edge_data[edge_data['t_label']==i]['time_score']
        re_et = [ "[" + x + "]" for x in et]
        edge_dict2[i] = list(re_et)

# ===== 2. 변환한 속성 merge ==== #
e_df = pd.DataFrame()
e_df['t_label'] = edge_dict.keys()
e_df['time_stamp'] = edge_dict.values()
e_df['time_score_stamp'] = edge_dict2.values()

# ===== 3. 타입변환 및 replace ==== #
# 1) time stamp
e_df['time_stamp'] = [ "<" + str(x) + ">" for x in e_df['time_stamp']]
e_df['time_stamp'] = [ x.replace("'","") for x in e_df['time_stamp']]
print(n_df['time_stamp'][0])

# 1) time socre stamp
e_df['time_score_stamp'] = [ str(x)[1:-2] for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ "<" + x + ">" for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ x.replace("'","") for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ x.replace(",",";") for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ x.replace("&",",") for x in e_df['time_score_stamp']]

print(e_df['time_score_stamp'][0])

# [3] index
n_df = n_df.sort_values(by='label')
n_df['id'] = range(len(n_df))

e_df['label'] = [x.split("&")[0] for x in e_df['t_label']]
e_df = pd.merge(e_df,n_df[['id','label']],how='left',on='label')
e_df.columns = ['t_label','time_stamp','time_score_stamp','Source_label','Source']

e_df['label'] = [x.split("&")[1] for x in e_df['t_label']]
e_df = pd.merge(e_df,n_df[['id','label']],how='left',on='label')
e_df.columns = ['t_label', 'time_stamp', 'time_score_stamp', 'Source_label', 'Source',
       'Target_label', 'target']

e_df = e_df.drop(['t_label'],axis=1)



<[2016]>
<[2017,2]; [2018,1]>


In [91]:
e_df.head()

,time_stamp,time_score_stamp,Source_label,Source,Target_label,target
0,"<[2017, 2018]>","<[2017,2]; [2018,1]>",B60N,114,G01D,283
1,"<[2016, 2017, 2018]>","<[2016,2]; [2017,6]; [2018,7]>",B60N,114,G01C,282
2,"<[2015, 2018]>","<[2015,3]; [2018,5]>",B60K,111,G07C,319
3,<[2015]>,"<[2015,1]>",F04B,237,F04D,239
4,<[2014]>,"<[2014,1]>",B04B,56,F16P,252


In [92]:
e_df.rename(columns={'time_score_stamp':'weight'},inplace=True)

In [93]:
e_df.head()

,time_stamp,weight,Source_label,Source,Target_label,target
0,"<[2017, 2018]>","<[2017,2]; [2018,1]>",B60N,114,G01D,283
1,"<[2016, 2017, 2018]>","<[2016,2]; [2017,6]; [2018,7]>",B60N,114,G01C,282
2,"<[2015, 2018]>","<[2015,3]; [2018,5]>",B60K,111,G07C,319
3,<[2015]>,"<[2015,1]>",F04B,237,F04D,239
4,<[2014]>,"<[2014,1]>",B04B,56,F16P,252


In [94]:
# save file
n_df.to_csv("AC_dyn_node2018_eric.csv",index=False)
e_df.to_csv("AC_dyn_edge2018_eric.csv",index=False)